## Defining cos
cos(x) = sum(x**n)
https://www.philipzucker.com/analysis_knuckle/


In [3]:
from z3 import *

B = BoolSort()
Z = IntSort()
R = RealSort()
x,y,z = Reals('x y z')

def lemma(thm, by=[], admit=False):
    if not admit:
        #print(Implies(And(by), thm).sexpr())
        prove(Implies(And(by), thm))
    else:
        print("Admitted!!", thm)
    return thm
    
plus = Function("plus", R,R,R)
plus_def = ForAll([x, y], plus(x, y) == x + y)

plus_0 = lemma(ForAll([x], plus(x, 0) == x), by=[plus_def])
plus_comm = lemma(ForAll([x, y], plus(x, y) == plus(y, x)), by=[plus_def])
plus_assoc = lemma(ForAll([x, y, z], plus(x, plus(y, z)) == plus(plus(x, y), z)), by=[plus_def])
                   
mul = Function("mul", R,R,R)
mul_def = ForAll([x, y], mul(x, y) == x * y)

mul_zero = lemma(ForAll([x], mul(x, 0) == 0), by=[mul_def])
mul_1 = lemma(ForAll([x], mul(x, 1) == x), by=[mul_def])
mul_comm = lemma(ForAll([x, y], mul(x, y) == mul(y, x)), by=[mul_def])
mul_assoc = lemma(ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)), by=[mul_def])
mul_distrib = lemma(ForAll([x, y, z], mul(x, plus(y, z)) == plus(mul(x, y), mul(x, z))), by=[mul_def, plus_def])  

Nat = Datatype("Nat")
Nat.declare('zero')
Nat.declare('succ', ('pred', Nat))
Nat = Nat.create()
n,m,k = Consts('n m k', Nat)

def induct(P):
    return Implies(And(P(Nat.zero), ForAll([n], Implies(P(n), P(Nat.succ(n))))),
                   #-----------------------------------------------------------
                   ForAll([n], P(n)))

real = Function("real", Nat, R)
real_def = ForAll([n], real(n) == If(Nat.is_zero(n), 0, plus(1, real(Nat.pred(n)))))

ceil = Function("ceil", R, Nat)
ceil_def = ForAll([x], ceil(x) == If(x <= 0, Nat.zero, Nat.succ(ceil(x-1))))

Seq = ArraySort(Nat, R)
sum_ = Function("sum", Seq, Nat, R)
s = Const('s', Seq)
sum_def = ForAll([s, n], sum_(s, n) == If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum_(s, Nat.pred(n)))))

abs = Function("abs", R, R)
abs_def = ForAll([x], abs(x) == If(x >= 0, x, -x))

proved
proved
proved
proved
proved
proved
proved
proved


In [ ]:
#https://en.wikipedia.org/wiki/Modulus_of_convergence
Modulus = ArraySort(R, Nat)
N = Const("N", Modulus)
eps = Real("eps")
cauchy = Function(Seq, Modulus, B)
cauchy_def == ForAll([s,N], cauchy(s, N) == ForAll([eps,n,m], Implies(And(n > N(eps), m > N(eps)), 
                                                                       abs(s[n] - s[m]) < eps)))


In [11]:
seq_sum = Function("seq_sum", Seq, Seq, Seq)
s,t = Consts("s t", Seq)
seq_sum_def = ForAll([s, t], seq_sum(s, t) == Lambda([n], plus(s[n], t[n])))

seq_zero = K(Nat, RealVal(0))
seq_zero.sort()
seq_sum_zero = lemma(ForAll([s], seq_sum(s, seq_zero) == s), by=[seq_sum_def, plus_def])




proved


In [ ]:
# Bundling as a record
# This is the reifnment problem.
CSeq = Datatype("CSeq")
CSeq.declare("mk", ("seq", Seq), ("mod", Modulus))
CSeq = CSeq.create()

cauchy = Function("cauchy", CSeq, B)
cauchy_def = ForAll([s], cauchy(s) == cauchy(s.seq, s.mod))


Implies(And(cauchy(M), cauchy(N)))


In [21]:
def myadd(x,y):
    match x.sort():
        case Seq:
            return seq_sum(x,y)
        case _:
            print(s.sort().name())
            assert False, s.sort().name()
# containers are going to be annoying
# "List" : Function("append", x.sort(), x.sort(), x.sort())(x,y)
ExprRef.__add__ = myadd
s + t

def mymul(x,y):
    match x.sort():
        case Seq:
            return seq_mul(x,y)
        case _:
            print(s.sort().name())
            assert False, s.sort().name()
# containers are going to be annoying
# "List" : Function("append", x.sort(), x.sort(), x.sort())(x,y)
ExprRef.__mul__ = mymul

def myneg(x):
    match x.sort():
        case ArraySort(R,R):
            z = FreshConst(R,prefix="z")
            return Lambda([z], -x[z])
ExprRef.__negate__ = myneg

x + y

In [22]:
#import multipledispatch # But we can't dispatch on the type.. so no go right?
add_dict = {
    Seq: seq_sum,
}
def myadd(x,y):
    return add_dict[x.sort()](x,y)
# what's the point of this indirection though.
def register_add(s, f):
    add_dict[s] = f

ExprRef.__add__ = myadd
s + t

seq_sum(s, t)

In [19]:
repr(s.sort())

'Array(Nat, Real)'

In [ ]:

lim = Function("lim", CSeq, R)
#lim_def = ForAll([s], lim(s) == If(cauchy(s), lim(s.seq), None))
lim_def = Implies(cauchy(s), conv_to(s.seq, lim(s), s.mod))

lim(s + t) == lim(s) + lim(t)
lim(s * t) == lim(s) * lim(t)


# if it converges
infsum = Function("infsum", Seq, R)

Implies(Implies(cauchy(f, Nf), cauchy(g, Ng)), cauchy(add(f,g), Lambda([eps], Nf(eps/2) + Ng(eps/2)))) 


#= cauchy(f,Nf), cauchy(g,Ng) => 
cauchy_eq = zero(sub(g,f), N)

In [ ]:
pow = Function("pow", R, Nat, R)
pow_def = ForAll([x,n], pow(x,n) == If(Nat.is_zero(n), RealVal(1), mul(x, pow(x,Nat.pred(n)))))




## Derivatives
Maybe we hsould have two notions of derivative. The analytic and syntactic version.
deriv() gives something
but
differentiable(f) => is_deriv(f,derive(f))

 https://www.philipzucker.com/z3_diff/

In [ ]:

RFun = ArraySort(R,R)
derive = Function("derive", RFun, RFun) #diff?
sin,cos,exp = Consts("sin cos exp", RFun)

dsin = derive(sin) == cos
dcos = derive(cos) == -sin
dexp = derive(exp) == exp

chain = ForAll([f,g], derive(comp(f,g)) == derive(g) * comp(derive(f),g))
dsum = ForAll([f,g], derive(f + g) == derive(f) + derive(g))
dmul = ForAll([f,g], derive(f * g) == derive(f) * derive(g))
c = Real("c")
dconst = ForAll([c], derive(K(R,c)) == K(R,0))
dx = derive(Lambda([x], x)) == K(R,1)

integ = Function("integ", RFun, RFun)
integ_def = ForAll([f], deriv(integ(f)) == f)
integ_deriv = ForAll([f], Implies(   , integ(deriv(f)) == f))






In [ ]:
M = DeclareSort("M") # manifold
RealField = ArraySort(M, R)
d = Function("d", RealField, ??)



## Float

Kahan summation 
https://news.ycombinator.com/item?id=40477604  Taming floating-point sums 

In [ ]:
x,y,z = Float("x y z")
subnormal
normal


lemma()


```z3
;re

; https://en.wikipedia.org/wiki/Machine_epsilon
(define-const eps32 Real (^ 2 -23))



; given eps > 0, abs-bound encodes |x| <= eps as conjunction of linear constraints
(define-fun abs-bound ((x Real) (eps Real)) Bool
    (and
        (<= x eps)
        (<= (- eps) x )
))

(declare-fun rnd-fun (Real) Real)

; Describing operations relationally is nice because you can instantiate properties at use site.
; We describe an overapproximation of the rounding operation
(define-fun rnd ((x Real) (res Real)) Bool
        (and (= res (rnd-fun x)) ; it is a functional relationship
            (abs-bound (- x res) (* eps32 (abs x))) ; with |x - rnd(x)| <= eps32 * |x|
        )
)

; floating point add is Real add then round.
(define-fun fp-add ((x Real) (y Real) (res Real)) Bool
    (rnd (+ x y) res)
)

(define-fun fp-mul ((x Real) (y Real) (res Real)) Bool
    (rnd (* x y) res)
)

; box is useful shorthand for stating initial conditions
(define-fun box ((x Real) (lower Real) (upper Real)) Bool
    (and (<= lower x)
         (<= x upper)
    )
)

(declare-const x Real)
(assert (box x 1 2))
(declare-const y Real)
(assert (box y 1 2))
(declare-const y Real)
(assert (box z 1 2))

(declare-const xy Real)
(assert (fp-add x y xy))
(declare-const xyy Real)
(assert (fp-add xy y xyy))

(declare-const xyz Real)
(assert (fp-mul xy z xyz))
(check-sat)
(get-model)
(eval (abs -1))

(push)
(assert-not (<= (- xyy (+ x y y)) 0.0001))
(check-sat)
(pop)

(push)
(assert-not (>= (- xyy (+ x y y)) -0.00001))
(check-sat)
(pop)

(push)
(assert-not (>= (- xyy (+ x y y)) -0.001))
(check-sat)
(pop)

;(get-model)
;(eval (- xyy (+ x y y)))


(define-fun myprog ((x Real) (y Real) (res Real)) Bool
    (exists ; define local variables
        ((xy Real) (fp73 Real))
     (and ; res := 7/3 * (x + y)
        (fp-add x y  xy)
        (rnd (/ 7 3) fp73)
        (fp-mul fp73 xy res)
     )

    )
)

(define-fun myprog-pre ((x Real) (y Real)) Bool

)

(define-fun myprog-post ((x Real) (y Real) (res Real))
    
)

(assert-not (forall ((x Real) (y Real) (res Real))
            (=> (myprog-pre x y)
                (and (myprog x y res) (myprog-post x y res))
            )
))
```

```
;re

(define-fun is-normal ((x Real)) Bool
    (or (and
            (< x (^ 2 300))
            (< (^ 2 -300) x)
         )
        (and
            (< (- x) (^ 2 300))
            (< (^ 2 -300) (- x))
        )
    )
)

(define-fun ulp ((x Real)) Real
   
)

(declare-fun rnd (Real) Real)

(assert (forall ((x Real)) 
    (=> (is-normal x)
         yada
    ) 

    (and (< (- (rnd x) x) (ulp x))
            (< (- (ulp x)) (- (rnd x) x) (ulp x))
))


(assert (forall ((x Real))
    (=>
        (and (< 1 x)
             (< x 2))
        (=  ulp(x))
    )
)


(define-fun-rec ulp ((x Real)) Real
    (ite (and (< x 1) (< x 2)))
         1
    (ite (x < 1)
        (/ (ulp (* x 2) 2)
    (ite (x > 2)
        (* 2 (ulp (/ x 2))
    )
)

;(define-fun fp-mul ((x Real) (y Real))
;    (rnd (* x y)))


; better as a relation?
; relations allow us to bundle in the lemmas we want
; without resorting to forall

; recipe: take a forall axiom, turn it into a define-fun
; This is kind of taking a forall and making it an axiom schema.
; You can still add the forall using (assert (forall ((x )) (my-pred x))
; but it gives the ability to explicitly instantiate

(define-fun rnd-ulp ((x Real)) Bool
    (and (< (- (rnd x) x) (ulp x))
        (< (- (ulp x)) (- (rnd x) x) (ulp x))
)




(define-fun rnd ((x Real) (res Real))
    (and 
        (= res (rnd-fun x))
        (rnd-ulp x)
    )
)


; that these are functions is almost a side feature
; Sometimes the functional nature is useful to the proof.
; probably not that often though

(define-fun ulp ((x Real) (y Real))
    (and
        (= y (ulp-fun x)))
)

(define-fun ulp-def ((x Real)) Bool

)

(define-fun fp-mul ((x Real) (y Real) (z Real))
    (let ((z1 (* x y)))
    (and 
        (= z z1)
        (rnd-ulp z1)
        (ulp-rel z1)

    )

(define-fun fp-sub ((x Real) (y Real) (res Real)) 
    (and
        ...
        (sterbenz x y)
    )

)

```

```z3
(define-sort Var () String)
(define-sort Env () (Array Var Int))
(declare-datatypes ()(
  (Term
    (Var (var1 Var))
    (Lit (lit1 Real))
    (Add (add1 Term) (add2 Term))
    (Mul (add1 Term) (add2 Term))
    (Sub (sub1 Term) (sub2 Term))
    ; And so on
  )
))

(define-fun-rec interp ((e Expr) (rho Env)) Bool
    (match e (
        ((Lit x) x)
        ((Var x) (select rho x))
        ((Add x y) (+ (interp x rho) (interp y rho))

    )
    
    )

)

; outer exists (exists ((rho Env))) is the same as (exists a b c d)
; So we don't get blocked going under an exists.

```

```python
from z3 import *
rnd = Function("rnd", RealSort(), RealSort)
class MyFloat():
    def __init__(self):
        self.x = rnd(FreshReal())
    def __add__(self,rhs):
        return rnd(self.x * rhs.x)

```

Klaus Weihrauch (2000), Computable Analysis.

## Interval


Flint has arb and calcium now folded in
<https://fredrikj.net/python-flint/>

```python
import flint
x = flint.arb(mid=7, rad=1e-10)
#dir(x)
flint.arb_poly([x, 1]).integral().derivative()
flint.arb_poly.interpolate([0, 1], [0, 1])
```


Use arb as knuckledragger style axiom schema.

What about user defined propagator in z3?

```python
def cos_schema(x):
    y = flint.cos(x)
    z = FreshReal()
    return ForAll([z], Implies(And(0 <= x. x <= ), And(y.lo <= cos(z), cos(z) <= y.hi)))

```

- <https://www.philipzucker.com/z3_diff/>
- <https://www.philipzucker.com/z3-cegar-interval/>

- Computational Functional Analysis by Moore <https://www.amazon.com/Computational-Functional-Analysis-Mathematics-Applications/dp/1904275249>
- Introduction to Interval Analysis  <https://epubs.siam.org/doi/10.1137/1.9780898717716>
- methods and applications of inverval analysis <https://epubs.siam.org/doi/book/10.1137/1.9781611970906>
- Interval Analysis: Application in the Optimal Control Problems
- Real Analysis: A Constructive Approach Through Interval Arithmetic - bridger
Interval Methods for Systems of Equations

bishop
Constructive functional analysis

```python
from z3 import *
def lemma(thm,by=[]):
    prove(Implies(And(by), thm))
    return thm

R = RealSort()
RR = ArraySort(R, R)

class RealFun(ArrayRef):
    def __add__(self, other):
        r = plus(self,other)
        r.__class__ = RealFun
        return r

def RealFuns(names):
    xs = Consts(names, RR)
    for x in xs:
        x.__class__ = RealFun
    return xs

z = Real("z")
f,g,h = RealFuns("f g h")
plus = Function("plus", RR, RR, RR)
plus_def = ForAll([f, g], plus(f, g) == Lambda([z], f[z] + g[z]))

plus_comm = lemma(ForAll([f, g], f + g == g + f), [plus_def])
plus_assoc = lemma(ForAll([f, g, h], f + g + h == f + (g + h)), [plus_def])

mul = Function("mul", RR, RR, RR)
mul_def = ForAll([f, g], mul(f, g) == Lambda([z], f[z] * g[z]))

mul_comm = lemma(ForAll([f, g], f * g == g * f), [mul_def])
mul_assoc = lemma(ForAll([f, g, h], f * g * h == f * (g * h)), [mul_def])

x, cos, sin = RealFuns("x cos sin")
x_def = x == Lambda([z],z)



integ = Function("integ", RR, R, R, R)





```python
cos.sort()
```

Array(Real, Real)

Interval analysis is a computational constructive framing

Moore Book
Bridger book
Errett Bishop

Kaucher intervals <https://arxiv.org/pdf/2202.03058>

Fixed point theorems.
An uninterpreted sort is not that bad for non recursive datatypes.

```python
from z3 import *

I = DeclareSort("Interval")
a, b, c = Consts("a b c", I)
R = RealSort()
x,y,z = Reals("x y z")
low = Function("low", I, R)
high = Function("high", I, R)
ForAll([a], low(a) <= high(a))

sub = Function("sub", I, I, BoolSort())
sub_def = ForAll([a,b], sub(a,b) == And(low(a) > low(b), high(a) < high(b)))

sub_irrefl = ForAll([a], Not(sub(a,a)))
prove(Implies(sub_def, sub_irrefl))

sub_asym = ForAll([a,b], Implies(sub(a,b), Not(sub(b,a))))
prove(Implies(sub_def, sub_asym))

sub_trans = ForAll([a,b,c], Implies(And(sub(a,b), sub(b,c)), sub(a,c)))
prove(Implies(sub_def, sub_trans))

subeq = Function("subeq", I, I, BoolSort())
subeq_def = ForAll([a,b], subeq(a,b) == Or(sub(a,b), a == b))

subeq_refl = ForAll([a], subeq(a,a))
prove(Implies(subeq_def, subeq_refl))

subeq_asym = ForAll([a,b], Implies(And(subeq(a,b), subeq(b,a)), a == b))
prove(Implies(And(subeq_def, sub_asym), subeq_asym))

subeq_trans = ForAll([a,b,c], Implies(And(subeq(a,b), subeq(b,c)), subeq(a,c)))
prove(Implies(And(subeq_def, sub_trans), subeq_trans))

# An interval can be dominated by another interval. This is distinct from subset
lt = Function("lt", I, I, BoolSort())
lt_def = ForAll([a,b], lt(a,b) == high(a) < low(b))


mono = Function("mono", ArraySort(I,I), BoolSort())
f = Array("f", I, I)
mono_def = ForAll([f], mono(f) == ForAll([a], sub(a,f[a])))

fix = Function("fix", ArraySort(I,I), I)
fix_def = ForAll([f], Implies(mono(f), fix(f) == f[fix(f)]))

# too dangerous to skolemize?
int_exists = ForAll([x,y], Implies( x <= y, Exists([a], And(low(a) == x, high(a) == y))))

iadd = Function("iadd", I, I, I)

iadd_comm = ForAll([x,y], iadd(x,y) == iadd(y,x))
iadd_assoc = ForAll([x,y,z], iadd(x,iadd(y,z)) == iadd(iadd(x,y),z))
iadd_mono = ForAll([x,y,z], Implies(sub(x,y), sub(iadd(x,z),iadd(y,z))))

iadd_high = ForAll([a,b], high(iadd(a,b)) == high(a) + high(b))
iadd_low = ForAll([a,b], low(iadd(a,b)) == low(a) + low(b))

iadd_monol = ForAll([a], mono(Lambda[a], iadd(a,b)))
```


```souffle

.type interval = [l : float, u : float]
.decl meet(x : interval , y : interval, z : interval) inline
meet([lx,ux],[ly,uy],[max(lx,ly), min(ux,uy)]) :- true.

.decl join(x : interval , y : interval, z : interval) inline
join([lx,ux],[ly,uy],[min(lx,ly), max(ux,uy)]) :- true.

.decl add(x : interval , y : interval, z : interval) inline
add([lx,ux],[ly,uy],[lx + ly, ux + uy]) :- true.

.decl sub(x : interval , y : interval, z : interval) inline
sub([lx,ux],[ly,uy],[lx - uy, ux - ly]) :- true.

.decl mul(x : interval , y : interval, z : interval) inline
mul([lx,ux],[ly,uy],[min(a,b,c,d), max(a,b,c,d)]) :- a = lx * ly, b = ux * ly, c = lx * uy, d = ux * uy.

.decl abs(x : interval , y : interval) inline
abs([lx,ux],[lx,ux]) :- lx > 0.
abs([lx,ux],[-ux, -lx]) :- ux < 0.
abs([lx,ux],[0, max(-lx, ux)]) :- lx <= 0, ux >= 0.


.decl test(x : interval)
test(z) :- mul([2,4], [4,5],z).
.output test(IO=stdout)


.type Expr = 
      Add {x : Expr, y : Expr}
    | Sub {x : Expr, y : Expr}
    | Mul {x : Expr, y : Expr}
    | Rnd {x : Expr}
    | Lit {x : float}
    | Var {x : symbol}

// Expression e is in interval i
.decl BND(e : Expr, i : interval)
// Absolute value of e is in interval i
.decl ABS(e : Expr, i : interval) 
// x = y * (1 + e) 
.decl REL(x : Expr, y : Expr, i : interval)
// 
//.decl FIX(x : Expr, k)

// The value of expression x is not zero.
.decl NZR(x : Expr)
.decl EQL(x : Expr, y : Expr) eqrel

EQL(x,y) :- BND(x, [a,a]), BND(y, [a,a]).
NZR(x) :- BND(x, [a,b]), (a > 0 ; b < 0).

.decl exprs(e : Expr)
EQL(x,x) :- exprs(x).

BND(t, [a,a]) :- exprs(t), t = $Lit(a).
BND(t, [0,0]) :- exprs(t), t = $Sub(a, a).

BND(z, iz) :- exprs(z), BND(x,ix), BND(y,iy), (
            (z = $Add(x,y), add(ix,iy,iz));
            (z = $Sub(x,y), sub(ix,iy,iz));
            (z = $Mul(x,y), mul(ix,iy,iz))).

ABS(e, iy) :- BND(e, ix), abs(ix,iy).


```



## Manifolds

title: Manifolds in Equational Logic


Automated reasoning over
Matroids. Synthetic theory of linear independence <https://en.wikipedia.org/wiki/Matroid>
universal injective
a dot b = a dot c !-> b = c

"represented by" =o=  observer dependent
two basic vectors e(0),e(1)
e'(1) e'(2) a different basis.

A parametrized set in abstract V
v : R R R -> V
v(20,30,-3)
It's a linear reprsentation / map
v(X1,Y1) + v(X2,Y2) = v(X1 + X2, Y1 + Y2)

dot(v(X,Y), v(W,Z)) = X*W + Y*Z

<https://en.wikipedia.org/wiki/Vector_space>


Interesting questions: Is a coordinate expression an invarinant?  f(X,Y,Z) ?= f'(p(X,Y,Z)). Converting coordinate expressions to invariant forms.
axiomatic minkowski

I don't know that there can be a canonical formulation of any logic.

But equational logic is trees of symbols, some of which are variable symbols. The 0-arity constant symbol `x` is very different from the variable `X` `?x`.
This distinction is useful. A statement `f(x) = 1` is a statement about the interaction of a particular `f` on a particular `x`. This may be an equation we'd like to solve for possibille values of `x`. `f(X) = 1` however is a universal statement. If we were working in forist order logic, which throws exists and forall quantifiers into the mix, this statement is the analog of `forall x, f_fol(x) =_fol 1_fol`. It is now stating nothing about a particular `x` but instead stating a global property of the function `f`, that `f` is the constant function.

Coordinate systems are a tricky concept. The notation is quite slippery and you learn your way around it.

Coordinates are an address book for points. They parametrize points such that.
In other words a coordinate system *is* a function of the type `R^n -> M`

The concept of a point is discussed in the textbooks, but it is an abstract entity. Since it doesn't really have calculational properties the way numbers, matrices, and numerical functions do. Even abstract things like quantum operators, kind of we have some matrix implementation in the back of my head. Points kind of have no operations on them in general. They can be in relation to other entities like curve, surfaces, etc, but there isn't a notion of point multiplication or addition or scaling except in very special spaces.

There is a related notion for a coordinate system of `x : M -> R`.

A coorindate transformation such as translation is written like `x' = x + a`. What are these `x`? This is certainly nonsensical as `X' = X + a`. It is also not exactly that we mean are talking about a particular `x` or `x'`. This might be so if we had a particular point of interest, `x'(p) = x(p) + a` but we are talking about transforming the coordinate *system* everywhere.
Actually what we are saying is that for any point `x'(P) = x(P) + a`.

Anyway, I think the appropriate notation here is

```
coord'(X,Y,Z) = coord(X - a, Y, Z)
```

Again, we get used to that if you have an expression `f(x')`,

```bash
cart : R^3 -> M
cart(_,_,_)

x y z : M -> R

x(cart(X,Y,Z)) = X
y(cart(X,Y,Z)) = Y

cyl(R,T,Z) = cart(Rcos(T), Rsin(T), Z)
cart(X,Y,Z) = cyl(sqrt(X^2 + Y^2), atan2(X,Y), Z)

point : M

dist : M -> M -> R
dist(P1,P2) 

% hyperreals?


a curve.
p : R -> M
p(T) = cart(T,T,0)


%vector derivative
% a rate of change of a function
dp : (M -> R) -> R -> R  % (?)
dp(f)(T) = f'(p(T)) * 2

% vector field
v : (M -> R) -> M -> R




a surface
q : M2 -> M

chart : R^2 -> M2
x(q(chart(X,Y))) =

% exterior deriavtive
d : (M -> R) -> M -> ?

% a different cartersian coordinate system
x(cart'(X,Y,Z)) = x(cart(X,Y,Z)) + a
```

it would be nice for the charts to be partial.
